# Cross-matching campaigns

In [4]:
# Parameters
api_url = '/services/data/v43.0/sobjects'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4", 
}
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [5]:
import pandas as pd
import datetime

#stagnant reports
import os
cwd = os.getcwd()

## SF pull all campaigns

In [6]:
# obtain list of all salesforce campaigns
from DB2SalesforceAPI import DB2SalesforceAPI
# obtain all campaign names
db_c = DB2SalesforceAPI(sf_login_params)
camp_names = db_c.query_data('SELECT Id, Name FROM Campaign')

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000SAKKwAAP
{"id":"7505w00000SAKKwAAP","operation":"query","object":"Campaign","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-23T18:55:53.000+0000","systemModstamp":"2020-11-23T18:55:54.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000SAKKwAAP","operation":"query","object":"Campaign","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-23T18:55:53.000+0000","systemModstamp":"2020-11-23T18:55:55.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":25,"retries":0,"totalProcessingTime":190}
[Success] Bulk job completed successfully.


In [4]:
display(camp_names.head(2))

,Id,Name
0,7015w000001c7B8AAI,Materials Science Engineering
1,7015w000001c7BDAAY,Citrine Webinar 10/21/2020


In [5]:
# standardize all names to convert dashes to underscores
camp_names['Name'] = camp_names['Name'].str.replace('-','_',regex=False)
display(camp_names.head(2))

,Id,Name
0,7015w000001c7B8AAI,Materials Science Engineering
1,7015w000001c7BDAAY,Citrine Webinar 10/21/2020


In [6]:
###### Important
camp_hierarchy = 'ML111120_'

## Importing Emails Sent

In [7]:
print(os.listdir(cwd+'/Emails Sent - Stagnant Reports'))

['C-A.xlsx', 'C-noR_noA.xlsx', 'C-Res_non_purdue.xlsx', 'C-Res_purdue.xlsx', 'C-R_noA.xlsx', 'Clo2d_sp_su_grA.xlsx', 'Clo2d_sp_su_grB.xlsx', 'L-A.xlsx', 'L-noR_noA.xlsx', 'L-R_noA.xlsx', 'U1d_fa_grA.xlsx', 'U1d_fa_grB.xlsx', 'U1d_sp_su_grA.xlsx', 'U1d_sp_su_grB.xlsx', 'U2_7d_fa_grA.xlsx', 'U2_7d_fa_grB.xlsx', 'U2_7d_sp_su_grA.xlsx', 'U2_7d_sp_su_grB.xlsx']


In [8]:
# match lists to campaigns
emails_sent = os.listdir(cwd+'/Emails Sent - Stagnant Reports')
for i,j in enumerate(emails_sent):
    emails_sent[i] = j.replace('-','_')
print(emails_sent)

['C_A.xlsx', 'C_noR_noA.xlsx', 'C_Res_non_purdue.xlsx', 'C_Res_purdue.xlsx', 'C_R_noA.xlsx', 'Clo2d_sp_su_grA.xlsx', 'Clo2d_sp_su_grB.xlsx', 'L_A.xlsx', 'L_noR_noA.xlsx', 'L_R_noA.xlsx', 'U1d_fa_grA.xlsx', 'U1d_fa_grB.xlsx', 'U1d_sp_su_grA.xlsx', 'U1d_sp_su_grB.xlsx', 'U2_7d_fa_grA.xlsx', 'U2_7d_fa_grB.xlsx', 'U2_7d_sp_su_grA.xlsx', 'U2_7d_sp_su_grB.xlsx']


In [9]:
# match email sent filenames to campaign names
sent_match_indexes = {} #key:index => index of email sent: compaign ID
for i,j in enumerate(emails_sent):
    j_temp = camp_hierarchy+j[:-5]
    if j_temp in camp_names['Name'].to_list():
        sent_match_indexes[i] = camp_names['Id'][camp_names['Name'].to_list().index(j_temp)]
    else:
        print(j)

In [10]:
print(sent_match_indexes)

{0: '7015w000001c7VNAAY', 1: '7015w000001c7VDAAY', 2: '7015w000001c7VmAAI', 3: '7015w000001c7VrAAI', 4: '7015w000001c7VIAAY', 5: '7015w000001c7VJAAY', 6: '7015w000001c7VKAAY', 7: '7015w000001c7V8AAI', 8: '7015w000001c7V3AAI', 9: '7015w000001c7V4AAI', 10: '7015w000001c7YwAAI', 11: '7015w000001c7VXAAY', 12: '7015w000001c7VSAAY', 13: '7015w000001c7Z1AAI', 14: '7015w000001c7VhAAI', 15: '7015w000001c7VdAAI', 16: '7015w000001c7VcAAI', 17: '7015w000001c7V5AAI'}


### Sequential Import

In [11]:
## Test ID transposing function
#t_input = '00Q5w00001tM5X6' #should be 00Q5w00001tM5X6EAK
import string
def ID_transpose(fault_digit,target_l=18):
    il = len(fault_digit)
    addon = ''
    if il == target_l:
        print('already good')
    else:
        for block in range(0,3,1):
            loop = 0
            for pos in range(0,5,1):
                current = fault_digit[block*5+pos]
                if current in list(string.ascii_uppercase):
                    loop += 1<<pos
            addon += "ABCDEFGHIJKLMNOPQRSTUVWXYZ012345"[loop]
    return fault_digit + addon
#print(ID_transpose(t_input))
#print(len(ID_transpose(t_input)))

In [12]:
#function email imports
def import_email_sent(filename,camp_id):
    #determine header and import data
    header_init = 0
    h_flag = False
    while h_flag == False:
        try:
            if 'L-' in filename or 'L_' in filename:
                opened_df = pd.read_excel(cwd+'/Emails Sent - Stagnant Reports/'+filename,header=header_init,\
                                      skipinitialspace=True, usecols=['Email','Lead ID']).dropna()
            else:
                opened_df = pd.read_excel(cwd+'/Emails Sent - Stagnant Reports/'+filename,header=header_init,\
                                      skipinitialspace=True, usecols=['Email','Contact ID']).dropna()
            h_flag = True
        except:
            header_init += 1
    print(header_init)
    
    ## Transposing 15 digit ID to 18 - https://help.salesforce.com/articleView?id=000319308&type=1&mode=1    
    try:
        if len(opened_df['Lead ID'][0]) != 18:
            opened_df['Lead ID'] = opened_df['Lead ID'].apply(lambda x: ID_transpose(x))
    except:
        try:
            if len(opened_df['Contact ID'][0]) != 18:
                opened_df['Contact ID'] = opened_df['Contact ID'].apply(lambda x: ID_transpose(x))
        except:
            print('Error in data - open the raw file')
    
    ##extract cols
    #contact vs leads
    if 'L-' in filename or 'L_' in filename: #then assume its a lead
        opened_df = opened_df[['Email','Lead ID']].rename(columns={'Email':'Email__c','Lead ID':'LeadId'})#.rename(columns={0:'Email__c'})
    else:
        opened_df = opened_df[['Email','Contact ID']].rename(columns={'Email':'Email__c','Contact ID':'ContactId'})#.rename(columns={0:'Email__c'})
    
    #concatenate
    ovr_df = opened_df
    
    #populate campaign name/id, campaign engagement
    ovr_df['CampaignId'] = camp_id
    ovr_df['Campaign_Engagement__c'] = 0 #placehold for replacement

    ovr_df.iloc[:opened_df.shape[0],-1] = 'Sent'
    ovr_df.iloc[opened_df.shape[0]:,-1] = 'Sent'
    
    return ovr_df

In [13]:
import time
#create enough DB2salesforceAPI objects to handle synchronous loading
db_e = []
from DB2SalesforceAPI import DB2SalesforceAPI
for i in range(len(sent_match_indexes.keys())):
    db_e.append(DB2SalesforceAPI(sf_login_params)) #emails
    db_e[i].object_id = 'CampaignMember'
    db_e[i].external_id = 'Email__c'

Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True


In [14]:
# process the data into SF compatible dataframes
# then upload data to SF
email_sent_original = os.listdir(cwd+'/Emails Sent - Stagnant Reports')
for ind,val in enumerate(list(sent_match_indexes.keys())):
    current_df = import_email_sent(email_sent_original[val],sent_match_indexes[val])
    current_df = current_df.drop_duplicates()
    ## upload to SF
    db_e[ind].send_data(current_df,print_flag=False)
    display(current_df.head(2))

11


,Email__c,ContactId,CampaignId,Campaign_Engagement__c
0,asoshnik@ucdavis.edu,0035w00003FSvfOAAT,7015w000001c7VNAAY,Sent
1,yongyang@ufl.edu,0035w00003FVt5EAAT,7015w000001c7VNAAY,Sent


12


,Email__c,ContactId,CampaignId,Campaign_Engagement__c
0,mahdi313@siu.edu,0035w00003EDffyAAD,7015w000001c7VDAAY,Sent
1,jbeiferm@asu.edu,0035w00003EDffdAAD,7015w000001c7VDAAY,Sent


16


...

,Email__c,ContactId,CampaignId,Campaign_Engagement__c
0,yudonghong1997@outlook.com,0035w00003FVjVpAAL,7015w000001c7VdAAI,Sent
1,walbercavs@gmail.com,0035w00003EDfg6AAD,7015w000001c7VdAAI,Sent


17


,Email__c,ContactId,CampaignId,Campaign_Engagement__c
0,19030707@students.ltu.edu.au,0035w000034K0qUAAS,7015w000001c7VcAAI,Sent
1,fathimarufaitha1828@gmail.com,0035w000034K0zoAAC,7015w000001c7VcAAI,Sent


17


,Email__c,ContactId,CampaignId,Campaign_Engagement__c
0,saurabhsharma.aue28@gmail.com,0035w000034JzYuAAK,7015w000001c7V5AAI,Sent
1,akw7uj@virginia.edu,0035w000034JzYxAAK,7015w000001c7V5AAI,Sent


## Importing Email Open Lists

In [15]:
# show list of all email open lists
print(os.listdir(cwd+'/Email Opens'))

['C-noR_noA.csv', 'C-noR_noA_ReminderOpen.csv', 'C-R_noA.csv', 'C-R_noA_ReminderOpen.csv', 'Clo2d_sp_su_grA.csv', 'Clo2d_sp_su_grA_ReminderOpen.csv', 'Clo2d_sp_su_grB.csv', 'Clo2d_sp_su_grB_ReminderOpen.csv', 'C_A.csv', 'C_A_ReminderOpen.csv', 'C_Res_non_purdue.csv', 'C_Res_non_purdue_ReminderOpen.csv', 'C_Res_purdue.csv', 'C_Res_purdue_ReminderOpen.csv', 'L-A.csv', 'L-A_ReminderOpen.csv', 'L-R_noA.csv', 'L-R_noA_ReminderOpen.csv', 'U1d_fa_grA.csv', 'U1d_fa_grA_ReminderOpen.csv', 'U1d_fa_grB.csv', 'U1d_fa_grB_ReminderOpen.csv', 'U1d_sp_su_grA.csv', 'U1d_sp_su_grA_ReminderOpen.csv', 'U1d_sp_su_grB.csv', 'U1d_sp_su_grB_ReminderOpen.csv', 'U2_7d_fa_grA.csv', 'U2_7d_fa_grA_ReminderOpen.csv', 'U2_7d_fa_grB.csv', 'U2_7d_fa_grB_ReminderOpen.csv', 'U2_7d_sp_su_grA.csv', 'U2_7d_sp_su_grA_ReminderOpen.csv', 'U2_7d_sp_su_grB.csv', 'U2_7d_sp_su_grB_ReminderOpen.csv']


### match lists to campaigns

In [16]:
email_opens = os.listdir(cwd+'/Email Opens')
for i,j in enumerate(email_opens):
    email_opens[i] = j.replace('-','_')
print(email_opens)

['C_noR_noA.csv', 'C_noR_noA_ReminderOpen.csv', 'C_R_noA.csv', 'C_R_noA_ReminderOpen.csv', 'Clo2d_sp_su_grA.csv', 'Clo2d_sp_su_grA_ReminderOpen.csv', 'Clo2d_sp_su_grB.csv', 'Clo2d_sp_su_grB_ReminderOpen.csv', 'C_A.csv', 'C_A_ReminderOpen.csv', 'C_Res_non_purdue.csv', 'C_Res_non_purdue_ReminderOpen.csv', 'C_Res_purdue.csv', 'C_Res_purdue_ReminderOpen.csv', 'L_A.csv', 'L_A_ReminderOpen.csv', 'L_R_noA.csv', 'L_R_noA_ReminderOpen.csv', 'U1d_fa_grA.csv', 'U1d_fa_grA_ReminderOpen.csv', 'U1d_fa_grB.csv', 'U1d_fa_grB_ReminderOpen.csv', 'U1d_sp_su_grA.csv', 'U1d_sp_su_grA_ReminderOpen.csv', 'U1d_sp_su_grB.csv', 'U1d_sp_su_grB_ReminderOpen.csv', 'U2_7d_fa_grA.csv', 'U2_7d_fa_grA_ReminderOpen.csv', 'U2_7d_fa_grB.csv', 'U2_7d_fa_grB_ReminderOpen.csv', 'U2_7d_sp_su_grA.csv', 'U2_7d_sp_su_grA_ReminderOpen.csv', 'U2_7d_sp_su_grB.csv', 'U2_7d_sp_su_grB_ReminderOpen.csv']


In [17]:
# match email open filenames to campaign names
match_indexes = {} #key:index => index of email_opens: compaign ID
for i,j in enumerate(email_opens):
    j_temp = camp_hierarchy+j[:-4]
    if j_temp in camp_names['Name'].to_list():
        match_indexes[i] = camp_names['Id'][camp_names['Name'].to_list().index(j_temp)]
    else:
        print(j)
## reminderOpen gets joined

C_noR_noA_ReminderOpen.csv
C_R_noA_ReminderOpen.csv
Clo2d_sp_su_grA_ReminderOpen.csv
Clo2d_sp_su_grB_ReminderOpen.csv
C_A_ReminderOpen.csv
C_Res_non_purdue_ReminderOpen.csv
C_Res_purdue_ReminderOpen.csv
L_A_ReminderOpen.csv
L_R_noA_ReminderOpen.csv
U1d_fa_grA_ReminderOpen.csv
U1d_fa_grB_ReminderOpen.csv
U1d_sp_su_grA_ReminderOpen.csv
U1d_sp_su_grB_ReminderOpen.csv
U2_7d_fa_grA_ReminderOpen.csv
U2_7d_fa_grB_ReminderOpen.csv
U2_7d_sp_su_grA_ReminderOpen.csv
U2_7d_sp_su_grB_ReminderOpen.csv


In [18]:
print(match_indexes)

{0: '7015w000001c7VDAAY', 2: '7015w000001c7VIAAY', 4: '7015w000001c7VJAAY', 6: '7015w000001c7VKAAY', 8: '7015w000001c7VNAAY', 10: '7015w000001c7VmAAI', 12: '7015w000001c7VrAAI', 14: '7015w000001c7V8AAI', 16: '7015w000001c7V4AAI', 18: '7015w000001c7YwAAI', 20: '7015w000001c7VXAAY', 22: '7015w000001c7VSAAY', 24: '7015w000001c7Z1AAI', 26: '7015w000001c7VhAAI', 28: '7015w000001c7VdAAI', 30: '7015w000001c7VcAAI', 32: '7015w000001c7V5AAI'}


### Sequential import

In [19]:
#function email imports
def import_email_open(filename,camp_id):
    reminder_fn = filename[:-4]+'_ReminderOpen.csv'
    #import data
    opened_df = pd.read_csv(cwd+'/Email Opens/'+filename)
    ropened_df = pd.read_csv(cwd+'/Email Opens/'+reminder_fn)
    
    ##extract cols
    #contact vs leads
    if 'L-' in filename or 'L_' in filename: #then assume its a lead
        opened_df = opened_df[['Subscriber Key','Email Address']].rename(columns={'Subscriber Key':'LeadId',\
                                                                              'Email Address':'Email__c'})
        ropened_df = ropened_df[['Subscriber Key','Email Address']].rename(columns={'Subscriber Key':'LeadId',\
                                                                              'Email Address':'Email__c'})
    else:
        opened_df = opened_df[['Subscriber Key','Email Address']].rename(columns={'Subscriber Key':'ContactId',\
                                                                              'Email Address':'Email__c'})
        ropened_df = ropened_df[['Subscriber Key','Email Address']].rename(columns={'Subscriber Key':'ContactId',\
                                                                              'Email Address':'Email__c'})
    #concatenate
    ovr_df = pd.concat([opened_df,ropened_df],ignore_index=True)
    
    #populate campaign name/id, campaign engagement
    ovr_df['CampaignId'] = camp_id
    ovr_df['Campaign_Engagement__c'] = 0 #placehold for replacement

    ovr_df.iloc[:opened_df.shape[0],-1] = 'Sent;Opened'
    ovr_df.iloc[opened_df.shape[0]:,-1] = 'Sent;Reminder Opened'
    
    return ovr_df

In [20]:
# process the data into SF compatible dataframes
# then upload data to SF
email_opens_original = os.listdir(cwd+'/Email Opens')
for ind,val in enumerate(list(match_indexes.keys())):
    current_df = import_email_open(email_opens_original[val],match_indexes[val])
    
    ## upload to SF
    db_e[ind].send_data(current_df,print_flag=False)
    display(current_df.head(2))

,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000034IrZVAA0,najes.riaz@hotmail.com,7015w000001c7VDAAY,Sent;Opened
1,0035w000034IsRyAAK,godguo@ucdavis.edu,7015w000001c7VDAAY,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000031Vut3AAC,shreyas.honrao@gmail.com,7015w000001c7VIAAY,Sent;Opened
1,0035w000031VzpjAAC,roberto@uniovi.es,7015w000001c7VIAAY,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000034JzozAAC,pjhen99@gmail.com,7015w000001c7VJAAY,Sent;Opened
1,0035w000034K0aPAAS,lowe77.purdue@gmail.com,7015w000001c7VJAAY,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000034IpwxAAC,alan5189100@gmail.com,7015w000001c7VKAAY,Sent;Opened
1,0035w000034IqwFAAS,yuchuw@umich.edu,7015w000001c7VKAAY,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000031VuidAAC,goranka.bilalbegovic@gmail.com,7015w000001c7VNAAY,Sent;Opened
1,0035w000031VzBxAAK,bishnoi@ieee.org,7015w000001c7VNAAY,Sent;Opened


...

,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000034Ip2wAAC,ooloba-whenu@unilag.edu.ng,7015w000001c7Z1AAI,Sent;Opened
1,0035w000034Ip2yAAC,elnajja2@illinois.edu,7015w000001c7Z1AAI,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w00003DBk2mAAD,luna.laviada@gmail.com,7015w000001c7VhAAI,Sent;Opened
1,0035w00003ECmdrAAD,bernacorak@gmail.com,7015w000001c7VhAAI,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w00003BRTdoAAH,gregkepner1@gmail.com,7015w000001c7VdAAI,Sent;Opened
1,0035w00003BRTkdAAH,jia00096@umn.edu,7015w000001c7VdAAI,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w000034IpJXAA0,roselin.raju@mbcet.ac.in,7015w000001c7VcAAI,Sent;Opened
1,0035w000034IpxRAAS,ritwik.das29@gmail.com,7015w000001c7VcAAI,Sent;Opened


,ContactId,Email__c,CampaignId,Campaign_Engagement__c
0,0035w00003BRUCbAAP,braishan@isu.edu,7015w000001c7V5AAI,Sent;Opened
1,0035w00003BRULqAAP,l.yitao@gmail.com,7015w000001c7V5AAI,Sent;Opened


# Cross referencing attendees and registrants with stagnant list

In [21]:
print(os.listdir(cwd+'/Emails Sent - Stagnant Reports'))
print(os.listdir(cwd+'/Registrants_Attendees'))

['C-A.xlsx', 'C-noR_noA.xlsx', 'C-Res_non_purdue.xlsx', 'C-Res_purdue.xlsx', 'C-R_noA.xlsx', 'Clo2d_sp_su_grA.xlsx', 'Clo2d_sp_su_grB.xlsx', 'L-A.xlsx', 'L-noR_noA.xlsx', 'L-R_noA.xlsx', 'U1d_fa_grA.xlsx', 'U1d_fa_grB.xlsx', 'U1d_sp_su_grA.xlsx', 'U1d_sp_su_grB.xlsx', 'U2_7d_fa_grA.xlsx', 'U2_7d_fa_grB.xlsx', 'U2_7d_sp_su_grA.xlsx', 'U2_7d_sp_su_grB.xlsx']
['ML111120Attendees.csv', 'ML111120Registrants.csv']


### Registrants cross referencing

In [22]:
# importing names for easy process
stag_reports = os.listdir(cwd+'/Emails Sent - Stagnant Reports')
ra_files = os.listdir(cwd+'/Registrants_Attendees')
sr_names = [val.replace('-','_') for val in stag_reports]

In [23]:
r_df = pd.read_csv(cwd+'/Registrants_Attendees/'+ra_files[-1], skipinitialspace=True, usecols=['Email',\
                                                            'Registered','FirstName','LastName'],encoding='cp1252')
#weird ass encoding - corrupts french and vietnamese characters, not sure wtf is going on
#pd.read_csv(cwd+'/Registrants_Attendees/'+ra_files[-1],encoding='UTF-8')#cp1252')
display(r_df.head(5))

,FirstName,LastName,Email,Registered
0,Ammar,Mohammed,ammarhussen659@gmail.com,Yes
1,Sergio,de Gioia,sergiodegioia@gmail.com,Yes
2,Sang-Jun,Lee,sangjun2@slac.stanford.edu,Yes
3,Enze,Chen,chenze@berkeley.edu,Yes
4,Giulia,Mancardi,giulia.mancardi@polito.it,Yes


In [24]:
# verify campaign ids
camp_names = db_c.query_data('SELECT Id, Name, ParentId FROM Campaign')
print(camp_names)

[Success] Bulk job creation successful. Job ID = 7505w00000S9j6sAAB
{"id":"7505w00000S9j6sAAB","operation":"query","object":"Campaign","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:23.000+0000","systemModstamp":"2020-11-20T17:59:23.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000S9j6sAAB","operation":"query","object":"Campaign","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:23.000+0000","systemModstamp":"2020-11-20T17:59:26.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":25,"retries":0,"totalProcessingTime":134}
[Success] Bulk job completed successfully.
                    Id                                 Name  \
0   7015w000001c7B8AAI 

In [25]:
# extract only those campaign IDs with relevant Names
needed_camps = []
for i,j in enumerate(camp_names['Name'].to_list()):
    if camp_hierarchy in j or j == 'External Invites':
        needed_camps.append(camp_names['Id'].to_list()[i])
print(needed_camps)

['7015w000001c7V3AAI', '7015w000001c7V4AAI', '7015w000001c7V5AAI', '7015w000001c7V8AAI', '7015w000001c7VDAAY', '7015w000001c7VIAAY', '7015w000001c7VJAAY', '7015w000001c7VKAAY', '7015w000001c7VNAAY', '7015w000001c7VSAAY', '7015w000001c7VXAAY', '7015w000001c7VcAAI', '7015w000001c7VdAAI', '7015w000001c7VhAAI', '7015w000001c7VmAAI', '7015w000001c7VrAAI', '7015w000001c7YwAAI', '7015w000001c7Z1AAI', '7015w000001c7d8AAA']


In [26]:
## match the list of registrants with the people in various stagnant reports 
## to determine which sub-campaign these people belong to
#query_member = 'select Email__c, Campaign_Engagement__c from CampaignMember where CampaignId = {}'.format(needed_camps[0])
query_member = 'select Email__c, Campaign_Engagement__c, CampaignId from CampaignMember'#
current_df = db_c.query_data(query_member)

# camp_names
display(current_df.head(5))
#print(current_df.head(2))

[Success] Bulk job creation successful. Job ID = 7505w00000S9j4TAAR
{"id":"7505w00000S9j4TAAR","operation":"query","object":"CampaignMember","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:35.000+0000","systemModstamp":"2020-11-20T17:59:35.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000S9j4TAAR","operation":"query","object":"CampaignMember","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:35.000+0000","systemModstamp":"2020-11-20T17:59:37.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":20124,"retries":0,"totalProcessingTime":1232}
[Success] Bulk job completed successfully.


,CampaignId,Campaign_Engagement__c,Email__c
0,7015w000001c7VNAAY,Sent;Opened,goranka.bilalbegovic@gmail.com
1,7015w000001c7VNAAY,Sent;Opened,bishnoi@ieee.org
2,7015w000001c7VNAAY,Sent;Opened,anwar@asu.edu
3,7015w000001c7VNAAY,Sent;Opened,tfaltens@purdue.edu
4,7015w000001c7VNAAY,Sent;Opened,hyunjae.lee@uwaterloo.ca


In [27]:
## pulls campaign information, and matches email addresses
def registrant_match(camp_id,reg_emails,cdf):
    ## find all people matches
    sf_emails = cdf['Email__c'].to_list()
    
    new_people = []
    for i,j in enumerate(reg_emails):
        if j in sf_emails:
            cdf.iloc[sf_emails.index(j),1]=';'.join([cdf.iloc[sf_emails.index(j),1],'Registered'])
        else:
            new_people.append(j)
            
    return cdf,new_people

In [28]:
r_emails = r_df['Email'].to_list()
from copy import deepcopy
cdf_updated_registrants,new_people = registrant_match(needed_camps[0],r_emails,deepcopy(current_df))
display(cdf_updated_registrants.head(2))
print(new_people)

,CampaignId,Campaign_Engagement__c,Email__c
0,7015w000001c7VNAAY,Sent;Opened,goranka.bilalbegovic@gmail.com
1,7015w000001c7VNAAY,Sent;Opened,bishnoi@ieee.org


[]


In [29]:
## for the new people/registrants create a Dataframe then pd.concat together
# identify the overarching campaign name - External Invites
###### Henry note - 7015w000001c7d8AAA/manual for now since no time
tempr_df = pd.DataFrame()
tempr_df['Email__c'] = new_people
tempr_df['CampaignId'] = '7015w000001c7d8AAA'
tempr_df['Campaign_Engagement__c'] = 'Registered'

In [30]:
# pull firstname and lastnames from r_df
tempr_df['FirstName'] = 0
tempr_df['LastName'] = 0
tempr_df['Company'] = '-'

for ind,val in enumerate(tempr_df['Email__c'].to_list()):
    tempr_df['FirstName'][ind] = r_df['FirstName'][r_df['Email'].to_list().index(val)]
    tempr_df['LastName'][ind] = r_df['LastName'][r_df['Email'].to_list().index(val)]
    

In [31]:
display(tempr_df.head(2))

,Email__c,CampaignId,Campaign_Engagement__c,FirstName,LastName,Company


### Create new lead upload for the new people and pull their new lead Ids

In [32]:
## It looks like these new people are leads
# create lead entries, then pull the created lead ID's back into the system
db_lead_upload = DB2SalesforceAPI(sf_login_params)
db_lead_upload.external_id = 'Email__c'
db_lead_upload.object_id = 'Lead'
db_lead_upload.send_data(tempr_df[['Email__c','FirstName','LastName','Company']])

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000S9j6xAAB
[Success] CSV upload successful. Job ID = 7505w00000S9j6xAAB
[Success] Closing job successful. Job ID = 7505w00000S9j6xAAB


In [33]:
db_lead_upload.check_bulk_status()

{'id': '7505w00000S9j6xAAB',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-11-20T17:59:51.000+0000',
 'systemModstamp': '2020-11-20T17:59:52.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Email__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [34]:
from pprint import pprint

pprint(db_lead_upload.check_bulk_failed_results())

''


In [35]:
# Pull latest IDs
newlead_df = db_lead_upload.query_data('select Id, Email__c from Lead')
display(newlead_df.head(2))

[Success] Bulk job creation successful. Job ID = 7505w00000S9j72AAB
{"id":"7505w00000S9j72AAB","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:53.000+0000","systemModstamp":"2020-11-20T17:59:53.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000S9j72AAB","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T17:59:53.000+0000","systemModstamp":"2020-11-20T17:59:55.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6940,"retries":0,"totalProcessingTime":572}
[Success] Bulk job completed successfully.


,Email__c,Id
0,NaN,00Q5w00001otOM1EAM
1,NaN,00Q5w00001otOM2EAM


In [36]:
# find email matches
tempr_df['LeadId'] = 0
for ind,val in enumerate(tempr_df['Email__c'].to_list()):
    tempr_df['LeadId'][ind] = newlead_df['Id'].to_list()[newlead_df['Email__c'].to_list().index(val)]
display(tempr_df.head(5))

,Email__c,CampaignId,Campaign_Engagement__c,FirstName,LastName,Company,LeadId


### Concat the separate registrations together

In [37]:
# join the dataframes together
registrants_df_ovr = pd.concat([cdf_updated_registrants,tempr_df],ignore_index=True,sort=True)
registrants_df_ovr = registrants_df_ovr.drop(columns=['Company','FirstName','LastName'])
display(registrants_df_ovr.head(5))

,CampaignId,Campaign_Engagement__c,Email__c,LeadId
0,7015w000001c7VNAAY,Sent;Opened,goranka.bilalbegovic@gmail.com,NaN
1,7015w000001c7VNAAY,Sent;Opened,bishnoi@ieee.org,NaN
2,7015w000001c7VNAAY,Sent;Opened,anwar@asu.edu,NaN
3,7015w000001c7VNAAY,Sent;Opened;Registered,tfaltens@purdue.edu,NaN
4,7015w000001c7VNAAY,Sent;Opened,hyunjae.lee@uwaterloo.ca,NaN


In [38]:
# send to SF
db_registrants =  DB2SalesforceAPI(sf_login_params)
db_registrants.external_id = 'Email__c'
db_registrants.object_id = 'CampaignMember'

Obtained Salesforce access token ...... True


In [39]:
db_registrants.send_data(registrants_df_ovr)

[Success] Bulk job creation successful. Job ID = 7505w00000S9j7MAAR
[Success] CSV upload successful. Job ID = 7505w00000S9j7MAAR
[Success] Closing job successful. Job ID = 7505w00000S9j7MAAR


In [40]:
# check status
db_registrants.check_bulk_status()

{'id': '7505w00000S9j7MAAR',
 'operation': 'upsert',
 'object': 'CampaignMember',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-11-20T18:00:06.000+0000',
 'systemModstamp': '2020-11-20T18:00:08.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Email__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [41]:
from pprint import pprint

pprint(db_registrants.check_bulk_failed_results())

''


## Attendees cross referencing

In [42]:
## note that attendees are in two locations:
## a) registrants include people who registered and and/or attended
## b) external attendees with no prior information

In [43]:
# importing names for easy process
ra_files = os.listdir(cwd+'/Registrants_Attendees')
print(ra_files)

['ML111120Attendees.csv', 'ML111120Registrants.csv']


In [44]:
#import the attendees from the registration file
r_df = pd.read_csv(cwd+'/Registrants_Attendees/'+ra_files[-1],skipinitialspace=True, usecols=['Email','Attended'])
display(r_df.head(2))

,Email,Attended
0,ammarhussen659@gmail.com,No
1,sergiodegioia@gmail.com,Yes


In [45]:
# only keep those that have attended
r_attend = [] #indexes
for ind,val in enumerate(r_df['Attended']):
    if val == 'Yes':
        r_attend.append(ind)
print(len(r_attend))

ar_df = pd.DataFrame(r_df.iloc[r_attend,0])
display(ar_df.shape)
display(ar_df.head(2))

175


(175, 1)

,Email
1,sergiodegioia@gmail.com
2,sangjun2@slac.stanford.edu


In [73]:
# importing attendees from attendees file
a_header = 0
a_import_flag = False
while a_import_flag == False and a_header <= 10:
    try:
        a_df = pd.read_csv(cwd+'/Registrants_Attendees/'+ra_files[0],enconding='cp1252',header=a_header,\
                       usecols=['Email','Name'])
        a_import_flag = True
    except:
        a_header += 1

display(a_df.head(2))

,Name,Email
0,Saaketh Desai,desai61@purdue.edu
1,Arun Kumar Mannodi Kanakkithodi,amannodi@purdue.edu


In [75]:
# concatenation
a_ovr_df = pd.concat([ar_df,a_df],ignore_index=True,sort=True)
display(a_ovr_df.head(2))
display(a_ovr_df.tail(2))

,Email,Name
0,sergiodegioia@gmail.com,NaN
1,sangjun2@slac.stanford.edu,NaN


,Email,Name
404,bahram5237@gmail.com,Bahram Rajabifar
405,oerturk@purdue.edu,Ozan Erturk


In [95]:
# pull latest information from SF
query_member = 'select Email__c, Campaign_Engagement__c, CampaignId from CampaignMember'#
current_df = db_c.query_data(query_member)

display(current_df.head(2))

[Success] Bulk job creation successful. Job ID = 7505w00000S9jUuAAJ
{"id":"7505w00000S9jUuAAJ","operation":"query","object":"CampaignMember","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T18:34:58.000+0000","systemModstamp":"2020-11-20T18:34:58.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000S9jUuAAJ","operation":"query","object":"CampaignMember","createdById":"0055w00000DM5bOAAT","createdDate":"2020-11-20T18:34:58.000+0000","systemModstamp":"2020-11-20T18:35:01.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":20124,"retries":0,"totalProcessingTime":1498}
[Success] Bulk job completed successfully.


,CampaignId,Campaign_Engagement__c,Email__c
0,7015w000001c7VNAAY,Sent;Opened,goranka.bilalbegovic@gmail.com
1,7015w000001c7VNAAY,Sent;Opened,bishnoi@ieee.org


In [96]:
# find matches
new_attendees = []
for ind,val in enumerate(a_ovr_df['Email'].to_list()):
    if val in current_df['Email__c'].to_list():
        current_df['Campaign_Engagement__c'][current_df['Email__c'].to_list().index(val)]+= ';Attended'
        current_df['Campaign_Engagement__c'][current_df['Email__c'].to_list().index(val)] = \
        ';'.join(list(dict.fromkeys(current_df['Campaign_Engagement__c'][current_df['Email__c'].to_list().index(val)].split(';'))))
    else:
        new_attendees.append(ind)
print(len(new_attendees))

0


In [97]:
display(current_df.head(2))
display(current_df.tail(2))

,CampaignId,Campaign_Engagement__c,Email__c
0,7015w000001c7VNAAY,Sent;Opened,goranka.bilalbegovic@gmail.com
1,7015w000001c7VNAAY,Sent;Opened,bishnoi@ieee.org


,CampaignId,Campaign_Engagement__c,Email__c
20122,7015w000001c7d8AAA,Registered;Registered,rong10@purdue.edu
20123,7015w000001c7d8AAA,Registered;Registered,kimbou@kaist.ac.kr


In [98]:
# create imports for new attendees - TBD
db_a = DB2SalesforceAPI(sf_login_params)
db_a.external_id = 'Email__c'
db_a.object_id = 'CampaignMember'
db_a.send_data(current_df)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000S9jV9AAJ
[Success] CSV upload successful. Job ID = 7505w00000S9jV9AAJ
[Success] Closing job successful. Job ID = 7505w00000S9jV9AAJ


In [105]:
# check status
db_a.check_bulk_status()

{'id': '7505w00000S9jV9AAJ',
 'operation': 'upsert',
 'object': 'CampaignMember',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-11-20T18:37:14.000+0000',
 'systemModstamp': '2020-11-20T18:37:55.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Email__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 20124,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 48417,
 'apiActiveProcessingTime': 42578,
 'apexProcessingTime': 70031}

In [106]:
from pprint import pprint

pprint(db_a.check_bulk_failed_results())

'"sf__Id","sf__Error",CampaignId,Campaign_Engagement__c,Email__c\n'
